In [1]:
import pandas as pd
structure = pd.read_csv("course-217-structure.csv")
data = pd.read_csv("course-217-events.csv")

In [2]:
import numpy as np
import numba

In [5]:
structure.head()

,course_id,module_id,module_position,lesson_id,lesson_position,step_id,step_position,step_type,step_cost
0,217,614,2,13228,2,38842,1,text,0
1,217,614,2,13228,2,39715,6,code,1
2,217,614,2,13228,2,39716,7,code,1
3,217,614,2,13228,2,39717,8,code,1
4,217,614,2,13228,2,39721,9,free-answer,0


In [6]:
pos_df = structure.ix[:, [-4, 2, 4, 6]].sort_values("module_position")

In [7]:
pos_df.head()

,step_id,module_position,lesson_position,step_position
223,38523,1,1,7
231,83925,1,1,13
219,38518,1,1,1
220,38519,1,1,3
222,38522,1,1,6


In [8]:
from tqdm import tqdm, tqdm_notebook

In [9]:
def get_next_step_id(cur_pos):
    if not pos_df.query("(module_position == {0}) & (lesson_position == {1}) & (step_position == {2} + 1)".format(*cur_pos)).empty:
        return pos_df.query("(module_position == {0}) & (lesson_position == {1}) & (step_position == {2} + 1)".format(*cur_pos)).step_id
    elif not pos_df.query("(module_position == {0}) & (lesson_position == {1} + 1) & (step_position == {2})".format(*cur_pos)).empty:
        return pos_df.query("(module_position == {0}) & (lesson_position == {1} + 1) & (step_position == {2})".format(*cur_pos)).step_id
    else:
        return pos_df.query("(module_position == {0} + 1) & (lesson_position == {1}) & (step_position == {2})".format(*cur_pos)).step_id

In [13]:
cur_pos = (1, 2, 7)
get_next_step_id(cur_pos)

2    39716
Name: step_id, dtype: int64

In [14]:
list(get_next_step_id(pos_df[pos_df.step_id == 38523].ix[:, [1,2,3]].values[0]).items())[0][1]

38524

In [15]:
def f(x):
    res = []
    for step_id in np.unique(x.step_id):
        first = second = third = False
        
        position = pos_df[pos_df.step_id == step_id].ix[:, [1,2,3]].values[0]
        try:
            next_step_id = list(get_next_step_id(position).items())[0][1]
        except:
            next_step_id = None
        if next_step_id:
            for cur_step_id in x.step_id:
                if first and second and third:
                    res.append(step_id)

                if cur_step_id == step_id and not first:
                    first = True
                if cur_step_id == next_step_id and not second:
                    second = True
                if cur_step_id == step_id and not third:
                    third = True
    return res

tqdm.pandas()
returnable_users = data.sort_values("time").groupby("user_id").progress_apply(f)

100%|██████████| 7663/7663 [37:04<00:00,  3.44it/s]


In [18]:
returnable_users = [np.unique(x) for x in returnable_users]

In [20]:
from collections import Counter
c = Counter()

for ar in returnable_users:
    c.update(ar)


In [32]:
coeffs = [(key, c[key] / len(data[data.step_id == key].groupby("user_id"))) for key in c]

In [33]:
res = sorted(coeffs, reverse=True, key=lambda x: x[1])[:10]
res
",".join([str(x[0]) for x in res])

[(47438, 0.9887459807073955),
 (44391, 0.9882943143812709),
 (38528, 0.9881586737714624),
 (39236, 0.9846857597454256),
 (48838, 0.9827089337175793),
 (48842, 0.9826589595375722),
 (47290, 0.9823204419889503),
 (42398, 0.9822161422708618),
 (38525, 0.9813411078717201),
 (38524, 0.980846581114729)]

'47438,44391,38528,39236,48838,48842,47290,42398,38525,38524'

In [31]:
len(data[data.step_id == 44120].groupby("user_id"))

1125